## Initial Setup

In [2]:
# Import necessary libraries
from io import StringIO
from kmodes.kmodes import KModes
import numpy as np
import pandas as pd
import requests
from scipy.special import logsumexp
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI, fowlkes_mallows_score as FMI
from sklearn.preprocessing import OneHotEncoder
import warnings

In [3]:
# Ignore all warnings
warnings.filterwarnings('ignore')

In [4]:
# Adjust the Pandas display options for better readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)

In [5]:
# List of datasets and their corresponding URLs
dataset_urls = {
    "Soybean (Small)": "https://archive.ics.uci.edu/ml/machine-learning-databases/soybean/soybean-small.data",
    "Zoo": "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data",
    "Heart Disease": "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
    "Breast Cancer Wisconsin (Original)": "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",
    "Dermatology": "https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data",
    "Letter Recognition (E, F)": "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data",
    "Molecular Biology (Splice-junction Gene Sequences)": "https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/splice-junction-gene-sequences/splice.data",
    "Mushroom": "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
}

In [6]:
# Set the true number of clusters for each dataset
n_clusters_dict = {
    "Soybean (Small)": 4,
    "Zoo": 7,
    "Heart Disease": 2,
    "Breast Cancer Wisconsin (Original)": 2,
    "Dermatology": 6,
    "Letter Recognition (E, F)": 2,
    "Molecular Biology (Splice-junction Gene Sequences)": 3,
    "Mushroom": 2,
}

In [7]:
# Set the number of runs for benchmarking
num_runs = 10

## Loading and Preprocessing of Datasets

In [9]:
dataframes = {}  # Dictionary to store cleaned dataframes

In [10]:
for name, url in dataset_urls.items():
    response = requests.get(url, verify=False)
    data = response.text
    
    # Convert the CSV/Text data into a DataFrame
    data_io = StringIO(data)
    df = pd.read_csv(data_io, header=None)
    
    df = df.replace('?', np.nan)  # Replace '?' with NaN
    df = df.dropna()  # Drop rows with NaN values

    # Set targets and features
    if name == "Letter Recognition (E, F)":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Mushroom":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Molecular Biology (Splice-junction Gene Sequences)":
        y = df.iloc[:, 0].str.strip()
        X = pd.DataFrame([list(seq.strip()) for seq in df.iloc[:, 2]])
    else:
        X, y = df.iloc[:, :-1], df.iloc[:, -1]

    # Drop columns with only 1 unique value
    for col in X.columns:
        if len(X[col].unique()) <= 1:
            X.drop(columns=[col], inplace=True) # Diregard warning as it is behaving as expected
    
    # Store in the dataframes dictionary
    dataframes[name] = {'features': X, 'targets': y}

In [11]:
def preprocess_datasets(dataframes):
    if 'Zoo' in dataframes:
        zoo_df = dataframes['Zoo']['features']
        zoo_df = zoo_df.drop(columns=[0])
        dataframes['Zoo']['features'] = zoo_df

    if 'Heart Disease' in dataframes:
        hd_df = dataframes['Heart Disease']['features']
        columns_to_drop = [0, 3, 4, 7, 9]
        hd_df = hd_df.drop(columns=hd_df.columns[columns_to_drop])
        dataframes['Heart Disease']['features'] = hd_df
        y_hd = dataframes['Heart Disease']['targets']
        dataframes['Heart Disease']['targets'] = y_hd.apply(lambda x: 0 if x == 0 else 1)
    
    if 'Breast Cancer Wisconsin (Original)' in dataframes:
        bcw_df = dataframes['Breast Cancer Wisconsin (Original)']['features']
        bcw_df = bcw_df.drop(columns=bcw_df.columns[0])
        dataframes['Breast Cancer Wisconsin (Original)']['features'] = bcw_df
    
    if 'Dermatology' in dataframes:
        derm_df = dataframes['Dermatology']['features']
        derm_df = derm_df.drop(columns=derm_df.columns[-1])
        dataframes['Dermatology']['features'] = derm_df

    if 'Letter Recognition (E, F)' in dataframes:
        lr_ef_df = dataframes['Letter Recognition (E, F)']['features']
        lr_ef_targets = dataframes['Letter Recognition (E, F)']['targets']
        mask = lr_ef_targets.isin(['E', 'F'])
        dataframes['Letter Recognition (E, F)']['features'] = lr_ef_df[mask]
        dataframes['Letter Recognition (E, F)']['targets'] = lr_ef_targets[mask]

    return dataframes

In [12]:
# Apply preprocessing to datasets
dataframes = preprocess_datasets(dataframes)

## Bernoulli Mixture Definition

In [14]:
class BernoulliMixture:
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    # Modified
    def get_save_single(self, x, mu):
        # Ensure x and mu are numpy arrays with float type
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        # Avoid taking log of zero by setting a minimum value
        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        # Perform the tensor dot product safely
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [15]:
def perform_bernoulli(features, n_components):
    """
    Perform clustering using the Bernoulli Mixture Model.
    Args:
        features (np.array): The input features for clustering.
        n_components (int): The number of components (clusters) in the mixture model.
    
    Returns:
        np.array: The predicted cluster labels for each sample.
    """
    # Initialize the Bernoulli Mixture model
    bm = BernoulliMixture(n_components=n_components, max_iter=1000, tol=1e-3)
    # Fit the model to the features
    bm.fit(features)
    # Predict the clusters
    cluster_labels = bm.predict(features)
    return cluster_labels

## Stochastic Bernoulli Mixture Definition

In [17]:
class StochasticBernoulliMixture:
    def __init__(self, n_components, max_iter, tol=1e-3, n_samples_per_component=10):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.n_samples_per_component = n_samples_per_component  # New parameter for SEM

    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # Stochastic E-step: draw a sample of latent variables and update gamma
            self.stochastic_e_step()
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log likelihood
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if abs(self.logL - self.old_logL) < self.tol:
                break
            if np.isnan(self.logL):
                self.reset_params()
                break

    def stochastic_e_step(self):
        # Drawing a sample of z based on current parameters
        log_probs = np.log(self.pi)[None, :] + self.get_log_bernoullis(self.x)
        log_probs -= logsumexp(log_probs, axis=1)[:, None]
        probs = np.exp(log_probs)
        self.gamma = np.zeros_like(probs)
        for i in range(self.x.shape[0]):
            for _ in range(self.n_samples_per_component):
                sampled_z = np.random.choice(self.n_components, p=probs[i])
                self.gamma[i, sampled_z] += 1
        self.gamma /= self.n_samples_per_component
        
    def get_log_bernoullis(self, x):
        # Compute the log probability for Bernoulli distribution
        return np.einsum('ij,kj->ik', x, np.log(self.mu)) + np.einsum('ij,kj->ik', 1-x, np.log(1-self.mu))
                
    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    def get_save_single(self, x, mu):
        # Ensure x and mu are numpy arrays with float type
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        # Avoid taking log of zero by setting a minimum value
        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        # Perform the tensor dot product safely
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        # Update mu based on the newly computed gamma
        self.mu = np.einsum('ik,id->kd', self.gamma, self.x) / np.sum(self.gamma, axis=0)[:, None]

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [18]:
def perform_stochastic_bernoulli(features, n_components):
    """
    Perform clustering using the Stochastic Bernoulli Mixture Model.
    Args:
        features (np.array): The input features for clustering.
        n_components (int): The number of components (clusters) in the mixture model.
    
    Returns:
        np.array: The predicted cluster labels for each sample.
    """
    # Initialize the Stochastic Bernoulli Mixture model with specified number of samples per component
    sbm = StochasticBernoulliMixture(n_components=n_components, max_iter=1000, tol=1e-3, n_samples_per_component=10)
    # Fit the model to the features
    sbm.fit(features)
    # Predict the clusters
    cluster_labels = sbm.predict(features)
    return cluster_labels

## KModes Definition

In [20]:
def perform_kmodes(features, n_clusters):
    """Perform clustering using KModes algorithm."""
    km = KModes(n_clusters=n_clusters, init='random', n_init=5)
    clusters = km.fit_predict(features)
    return clusters

## Execution of Clustering Algorithms

In [22]:
def run_clustering_algorithms(dataframes, n_clusters_dict, num_runs=10):
    results_list = []
    for name, data in dataframes.items():
        print("Processing:", name)
        
        features = data['features']
        true_labels = data['targets'].squeeze()  # Assuming targets are in a single column
        n_clusters = n_clusters_dict.get(name, 2)  # Default to 2 clusters if not specified
        
        encoder = OneHotEncoder(sparse=False)
        features_encoded = encoder.fit_transform(features)

        metrics = {'Bernoulli Mixture': [], 'Stochastic Bernoulli Mixture': [], 'KModes': []}  # Initialize a dictionary to store results for each method
        
        for _ in range(num_runs):
            # Bernoulli Mixture
            bm_clusters = perform_bernoulli(features_encoded, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, bm_clusters)
            metrics['Bernoulli Mixture'].append((ari, nmi, fmi))
            
            # Stochastic Bernoulli Mixture
            sbm_clusters = perform_stochastic_bernoulli(features_encoded, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, sbm_clusters)
            metrics['Stochastic Bernoulli Mixture'].append((ari, nmi, fmi))
            
            # KModes
            km_clusters = perform_kmodes(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, km_clusters)
            metrics['KModes'].append((ari, nmi, fmi))

        # Calculate mean and standard deviation for each method and append to results list
        for method, values in metrics.items():
            ari_vals, nmi_vals, fmi_vals = zip(*values)
            ari_mean, ari_std = np.mean(ari_vals), np.std(ari_vals)
            nmi_mean, nmi_std = np.mean(nmi_vals), np.std(nmi_vals)
            fmi_mean, fmi_std = np.mean(fmi_vals), np.std(fmi_vals)
            results_list.append({
                "Dataset": name,
                "Method": method,
                "ARI": f"{ari_mean:.4f}±{ari_std:.2f}",
                "NMI": f"{nmi_mean:.4f}±{nmi_std:.2f}",
                "FMI": f"{fmi_mean:.4f}±{fmi_std:.2f}"
            })

    # Convert list of dictionaries to DataFrame for results
    results_df = pd.DataFrame(results_list)
    return results_df

In [23]:
def calculate_metrics(true_labels, predicted_labels):
    """Calculate clustering metrics: Adjusted Rand Index, Normalized Mutual Information, and Folkes-Mallows Index."""
    return ARI(true_labels, predicted_labels), NMI(true_labels, predicted_labels), FMI(true_labels, predicted_labels)

In [24]:
# Running all algorithms and storing the results
results = run_clustering_algorithms(dataframes, n_clusters_dict, num_runs)

Processing: Soybean (Small)
Processing: Zoo
Processing: Heart Disease
Processing: Breast Cancer Wisconsin (Original)
Processing: Dermatology
Processing: Letter Recognition (E, F)
Processing: Molecular Biology (Splice-junction Gene Sequences)
Processing: Mushroom


## Presentation of Results

In [26]:
def reformat_results(results_df):
    # Expanding 'ARI', 'NMI', and 'FMI' columns into multiple rows with a new 'Metric' column
    expanded_df = pd.melt(results_df, id_vars=["Dataset", "Method"], value_vars=["ARI", "NMI", "FMI"], var_name="Metric", value_name="Value")
    expanded_df[['Metric_Value', 'Std']] = expanded_df['Value'].str.split('±', expand=True)
    expanded_df.drop(columns=['Value'], inplace=True)  # Removing the original combined column
    
    # Convert the 'Metric_Value' and 'Std' columns to numeric types
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].astype(float)
    expanded_df['Std'] = expanded_df['Std'].astype(float)

    # Concatenate the metric value and standard deviation back into a single string
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].map('{:.4f}'.format) + "±" + expanded_df['Std'].map('{:.2f}'.format)
    
    # Ensuring the order of datasets and methods remains consistent with the original DataFrame
    dataset_order = results_df['Dataset'].unique()
    method_order = results_df['Method'].unique()

    # Creating a pivot table to restructure the DataFrame as required
    pivot_df = expanded_df.pivot_table(index=["Dataset", "Metric"], columns="Method", values="Metric_Value", aggfunc='first')
    
    # Reindexing the pivot table to maintain the original order
    pivot_df = pivot_df.reindex(dataset_order, level='Dataset')
    pivot_df = pivot_df.reindex(method_order, axis='columns')

    return pivot_df

In [27]:
# Reformat the results
formatted_results = reformat_results(results)

In [28]:
# Print the formatted results
print(formatted_results)

Method                                                    Bernoulli Mixture Stochastic Bernoulli Mixture       KModes
Dataset                                            Metric                                                            
Soybean (Small)                                    ARI          1.0000±0.00                  0.9356±0.09  0.9143±0.07
                                                   FMI          1.0000±0.00                  0.9546±0.06  0.9355±0.05
                                                   NMI          1.0000±0.00                  0.9543±0.05  0.9376±0.04
Zoo                                                ARI          0.6972±0.00                  0.6515±0.06  0.6784±0.12
                                                   FMI          0.7645±0.00                  0.7282±0.05  0.7497±0.09
                                                   NMI          0.8028±0.00                  0.7790±0.02  0.7929±0.05
Heart Disease                                      ARI  

### Performance Comparison and Explanation

#### General Observations

- The Bernoulli Mixture generally performs consistently well across all three metrics, often achieving the highest or near-highest scores.
- The Stochastic Bernoulli Mixture is competitive, often close to Bernoulli Mixture, indicating a slight variation due to its stochastic nature.
- KModes shows varying performance, performing comparably in simpler datasets but lagging in more complex or larger datasets.

#### Why is FMI consistently higher than ARI and NMI?

- **FMI** measures the geometric mean of precision and recall, focusing on the number of correct decisions (true positive) over all pairs. It is generally less sensitive to the actual grouping (cluster purity and recovery), which can make it appear higher as it emphasizes agreement over all pairs.
- **ARI** adjusts for chance in the clustering performance, providing a more stringent measure that evaluates both the true positive and negative decisions against possible random chance. This can result in lower scores if the clustering contains many splits or mergers not aligned with true labels.
- **NMI** evaluates mutual information in relation to the clusters' and ground truth's entropies, adjusted for chance. It's sensitive to the actual amount of shared information between the clustering result and the true labels, which can be lower if clusters are not perfectly informative regarding true groupings.

#### Why are ARI and NMI often lower compared to FMI?

- Both ARI and NMI are more sensitive to the actual alignment and information shared between the predicted clusters and the true labels. They penalize both over-segmentation and under-segmentation more significantly than FMI. ARI, being adjusted for chance, reflects a more conservative estimate of clustering quality, often resulting in lower scores especially when clusters are not well-separated or are uneven. NMI similarly reflects discrepancies in informational alignment, which can result in lower values if clustering does not capture the natural groups effectively.

#### Dataset Complexity and Clustering Challenges

- Datasets like "Soybean (Small)" and "Breast Cancer Wisconsin" with high scores across methods suggest that these datasets have clear, distinguishable clusters that all methods can identify well.
- Datasets with mixed or lower scores, like "Heart Disease" or "Molecular Biology," suggest more complex data structures or less distinct clustering patterns, challenging for methods, especially non-stochastic ones.